test file for nlp packages that predict race to see acuuracy,the following are the packages using 

1. ethnicolr
The model uses US census data, the Florida voting registration data, and the Wikipedia data collected by Skiena and colleagues, to predict race and ethnicity based on first and last name or just the last name. 
The model provides 06 functions. 

Source: https://ethnicolr.readthedocs.io/ethnicolr.html#install  



The Prediction Process: 

owner ----> nlp ----> W: 70% NW: 30%    
block group ... ----> W: 80% NW: 20%
            .
            .
            .
            .
            and use
        min & max scale? 
          ex.  
            1 -----> 40

In [1]:
import pandas as pd

#load data fram with names here
#this is the data frame we will use
AtoZ_df = pd.read_csv('../data/mergent_intellect_executives/AtoZ_executive_flagged.csv.xz')

AtoZ_df

,ID,company_name,fullname,title,firstname,lastname,flag_executive,flag_owner
0,2,Capital Jiu-Jitsu Team/Royce,Jeremiah Lafreniere,Owner,Jeremiah,Lafreniere,1,1
1,3,Integrated Clinical Concepts,Judith Wald,Manager,Judith,Wald,1,0
2,4,Magical Elves Inc,Ben Mack,Manager,Ben,Mack,1,0
3,5,Potomac River Running Store,Chris Jakubek,Manager,Chris,Jakubek,1,0
4,7,Atlantic Sweeping & Cleaning Inc,Bonita Tenicela,Controller,Bonita,Tenicela,1,0
...,...,...,...,...,...,...,...,...
29871,50399,Richard A Spagna,Richard Spagna,Manager,Richard,Spagna,1,0
29872,50400,Kathy Wolf MD PC,Kathy Wolf,Principal,Kathy,Wolf,1,0
29873,50401,Roger F Vorcheimer DDS,Roger Vorcheimer,Professional,Roger,Vorcheimer,1,0
29874,50402,Richard F Blackburn,Richard Blackburn,Manager,Richard,Blackburn,1,0


# Ethnicolor:

In [2]:
import ethnicolr
from ethnicolr import census_ln, pred_census_ln

namesdf = pd.DataFrame(AtoZ_df, columns=['lastname', 'firstname'])
#the orignal data frame(mergent_executives_flagged.csv.xz) contains 29,876 names
#this is so i don't have to write it out
f = 'firstname'
l = 'lastname'
namesdf #dataframe with only first and last names

#-----------IMPORTANT----------
#1. for binary classification dfs and columns:
#I added a prop not_white column and a race column using below boolean values:
#1 if minority 
#0 if white

#2. I added a computation under each df to describe the number of minority guesses.. i also created
#a list of all the executive names that were flagged a minority if that helps in any way

2023-07-26 14:59:02.489296: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-26 14:59:02.797999: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-26 14:59:02.798820: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 14:59:05.960135: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


,lastname,firstname
0,Lafreniere,Jeremiah
1,Wald,Judith
2,Mack,Ben
3,Jakubek,Chris
4,Tenicela,Bonita
...,...,...
29871,Spagna,Richard
29872,Wolf,Kathy
29873,Vorcheimer,Roger
29874,Blackburn,Richard


## pred_census_ln w firstnames

In [3]:
fdf = pred_census_ln(namesdf, f, 2010)

#sort to binary classification w first names

d = pd.DataFrame(fdf, columns=[f, 'white'])
d['not_white']= 1-d['white']


racelist = list(fdf['race'])
notw = list(d['not_white']) 
race = []

for n in notw:
    if n > .50:
        #print(True)
        race.append(1)
    else:
        #print(False)  
        race.append(0)

race
d['race'] = race
d

#I added a prop not_white column and made it so that the race column used boolean values:
#1 if minority 
#0 if white

2023-07-26 14:59:11.632207: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


,firstname,white,not_white,race
0,Jeremiah,0.386579,0.613421,1
1,Judith,0.854711,0.145289,0
2,Ben,0.348896,0.651104,1
3,Chris,0.698235,0.301765,0
4,Bonita,0.547929,0.452071,0
...,...,...,...,...
6296,Maribell,0.801131,0.198869,0
6297,Yeun-Hee,0.410363,0.589637,1
6298,Bayani,0.100641,0.899359,1
6299,Raven,0.685576,0.314424,0


## pred_census_ln w lastnames

In [4]:
ldf = pred_census_ln(namesdf, l, 2010)
#sort into binary classification with last names

dd = pd.DataFrame(ldf, columns=[l, 'white'])
notw = list(1-dd['white'])
dd['not_white'] = notw

race = list(list(ldf['race']))

newlist = []
for r in race:
    newlist.append(str(r))
newlist

correct = []
for n in list(dd['not_white']):
    if .50 < n < 1.00:
        #print(True)
        correct.append(1)
    else:
        #print(False)  
        correct.append(0)


dd['race'] = correct
dd


,lastname,white,not_white,race
0,Lafreniere,0.923821,0.076179,0
1,Wald,0.906037,0.093963,0
2,Mack,0.415999,0.584001,1
3,Jakubek,0.989522,0.010478,0
4,Tenicela,0.036939,0.963061,1
...,...,...,...,...
4547,Avedisian,0.715930,0.284070,0
4548,Waheed,0.497597,0.502403,1
4549,Vann,0.705145,0.294855,0
4550,Henderson,0.641722,0.358278,0


In [5]:
print(correct.count(1), len(race))
#1,376 minority predictions

1376 4552


## pred_wiki_name (this takes both the first and last names)

In [6]:
from ethnicolr import pred_wiki_name
wikifulldf = pred_wiki_name(namesdf,l, f, conf_int=0.9)

white = ['GreaterEuropean,WestEuropean,Italian_mean', 'GreaterEuropean,WestEuropean,Nordic_mean', 
          'GreaterEuropean,WestEuropean,Germanic_mean', 'GreaterEuropean,WestEuropean,French_mean',
          'GreaterEuropean,Jewish_mean', 'GreaterEuropean,British_mean','GreaterEuropean,EastEuropean_mean']
whitelist = ['GreaterEuropean,WestEuropean,Italian', 'GreaterEuropean,WestEuropean,Nordic',
         'GreaterEuropean,WestEuropean,Germanic', 'GreaterEuropean,WestEuropean,French',
          'GreaterEuropean,Jewish', 'GreaterEuropean,British', 'GreaterEuropean,EastEuropean']

#sort into binary classification
a = pd.DataFrame(wikifulldf, columns=[f, l])
a['white'] = wikifulldf[white].sum(axis=1)
a['not_white'] = 1- a['white']
a['race'] = wikifulldf['race']

race = []

for n in list(a['not_white']):
    if .50 < n < 1.00:
        #print(True)
        race.append(1)
    else:
        #print(False)  
        race.append(0)
 

a['race'] = race

a
#null values:
#defalut is 1.00 not_white and for thre race column thats automatcially gunna be 0

['Asian,GreaterEastAsian,EastAsian', 'Asian,GreaterEastAsian,Japanese', 'Asian,IndianSubContinent', 'GreaterAfrican,Africans', 'GreaterAfrican,Muslim', 'GreaterEuropean,British', 'GreaterEuropean,EastEuropean', 'GreaterEuropean,Jewish', 'GreaterEuropean,WestEuropean,French', 'GreaterEuropean,WestEuropean,Germanic', 'GreaterEuropean,WestEuropean,Hispanic', 'GreaterEuropean,WestEuropean,Italian', 'GreaterEuropean,WestEuropean,Nordic']


,firstname,lastname,white,not_white,race
0,Jeremiah,Lafreniere,0.958963,0.041037,0
1,Judith,Wald,0.953411,0.046589,0
2,Ben,Mack,0.951360,0.048640,0
3,Chris,Jakubek,0.543747,0.456253,0
4,Bonita,Tenicela,0.785190,0.214810,0
...,...,...,...,...,...
4547,Ara,Avedisian,0.000000,1.000000,0
4548,Sajal,Waheed,0.000000,1.000000,0
4549,Maribell,Vann,0.000000,1.000000,0
4550,Raven,Henderson,0.000000,1.000000,0


In [7]:
print(race.count(1), len(race))
#619 minority predictions

617 4552


##  pred_fl_reg_name(df, 'last', 'first', conf_int=0.9) w fullname

In [8]:
from ethnicolr import pred_fl_reg_name, pred_fl_reg_name_five_cat
odf = pred_fl_reg_name(namesdf, l, f, conf_int=0.9)

#sort binary
b = pd.DataFrame(odf, columns=[f, l])
b['white'] = odf['nh_white_mean']
b['not_white'] = 1 - b['white']
b['race'] = odf['race']


race = []

        

for n in list(b['not_white']):
    if n > .50:
        #print(True)
        race.append(1)
    else:
        #print(False)  
        race.append(0)
 

b['race'] = race
b

['asian', 'hispanic', 'nh_black', 'nh_white']


,firstname,lastname,white,not_white,race
0,Jeremiah,Lafreniere,0.607134,0.392866,0
1,Judith,Wald,0.960800,0.039200,0
2,Ben,Mack,0.824022,0.175978,0
3,Chris,Jakubek,0.882107,0.117893,0
4,Bonita,Tenicela,0.764582,0.235418,0
...,...,...,...,...,...
4547,Ara,Avedisian,NaN,NaN,0
4548,Sajal,Waheed,NaN,NaN,0
4549,Maribell,Vann,NaN,NaN,0
4550,Raven,Henderson,NaN,NaN,0


In [9]:
print(race.count(1), len(race))
# 588 minority predictions

595 4552


## pred_fl_reg_name_five_cat(df, namecol, num_iter=100, conf_int=1.0) w/full name input

In [10]:
rdf = pred_fl_reg_name_five_cat(namesdf, l, f, num_iter=100, conf_int=0.9)
#sort to binary
c = pd.DataFrame(rdf, columns=[f, l])
c['white'] = rdf['nh_white_mean']
c['not_white'] = 1 - c['white']
c['race'] = rdf['race']

racelist = list(c['race'])
race = []

        

for n in list(c['not_white']):
    if n > .50:
        #print(True)
        race.append(1)
    else:
        #print(False)  
        race.append(0)
 

c['race'] = race
c

['asian', 'hispanic', 'nh_black', 'nh_white', 'other']


,firstname,lastname,white,not_white,race
0,Jeremiah,Lafreniere,0.084253,0.915747,1
1,Judith,Wald,0.609208,0.390792,0
2,Ben,Mack,0.320544,0.679456,1
3,Chris,Jakubek,0.336253,0.663747,1
4,Bonita,Tenicela,0.287175,0.712825,1
...,...,...,...,...,...
4547,Ara,Avedisian,NaN,NaN,0
4548,Sajal,Waheed,NaN,NaN,0
4549,Maribell,Vann,NaN,NaN,0
4550,Raven,Henderson,NaN,NaN,0


In [11]:
print(race.count(1), len(race))
# 1361 minority predictions

1358 4552


## pred_nc_reg_name(df, namecol, num_iter=100, conf_int=1.0) w/ full names 

In [12]:
from ethnicolr import pred_nc_reg_name
z = pred_nc_reg_name(namesdf, l, f, num_iter=100, conf_int=.90)
# / ***************************************************************************
# Race codes
# race               description
# *******************************************************************************
# A                  ASIAN
# B                  BLACK or AFRICAN AMERICAN
# I                  AMERICAN INDIAN or ALASKA NATIVE
# M                  TWO or MORE RACES
# O                  OTHER
# P                  NATIVE HAWAIIAN or PACIFIC ISLANDER
# U                  UNDESIGNATED
# W                  WHITE
# *************************************************************************** /

# / ***************************************************************************
# Ethnic codes
# ethnicity          description
# *******************************************************************************
# HL                 HISPANIC or LATINO
# NL                 NOT HISPANIC or NOT LATINO
# UN                 UNDESIGNATED
# *************************************************************************** /

2023-07-26 15:00:53.056842: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-26 15:00:53.058294: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-26 15:00:53.059310: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

['HL+A', 'HL+B', 'HL+I', 'HL+M', 'HL+O', 'HL+W', 'NL+A', 'NL+B', 'NL+I', 'NL+M', 'NL+O', 'NL+W']


In [13]:
#sort into binary classification df
e = pd.DataFrame(a, columns=[f, l])
e['white'] =  z['NL+W_mean'] + z['HL+W_mean']
e['not_white'] = 1 - e['white']
e['race'] = z['race']

race = []

        

for n in list(e['not_white']):
    if n > .50:
        #print(True)
        race.append(1)
    else:
        #print(False)  
        race.append(0)

e['race'] = race
e

,firstname,lastname,white,not_white,race
0,Jeremiah,Lafreniere,0.770279,0.229721,0
1,Judith,Wald,0.383571,0.616429,1
2,Ben,Mack,0.417455,0.582545,1
3,Chris,Jakubek,0.366039,0.633961,1
4,Bonita,Tenicela,0.486375,0.513625,1
...,...,...,...,...,...
4547,Ara,Avedisian,NaN,NaN,0
4548,Sajal,Waheed,NaN,NaN,0
4549,Maribell,Vann,NaN,NaN,0
4550,Raven,Henderson,NaN,NaN,0


In [14]:
print(race.count(1), len(race))
#1294 minority predictions

1294 4552


# Done with testing
## Adding probability columns that an owner is not white to the original data frame

In [15]:
AtoZ_df['prob_nw_eth_wiki'] = a['not_white']
AtoZ_df['prob_nw_eth_fl_reg_name'] = b['not_white']
AtoZ_df['prob_nw_eth_fl_five_cat'] = c['not_white']
AtoZ_df['prob_nw_eth_census_fn'] = d['not_white']
AtoZ_df['prob_nw_eth_census_ln'] = dd['not_white']
AtoZ_df['prob_nw_eth_nc'] = e['not_white']
# nonull = AtoZ_df.dropna() 
# nonull
AtoZ_df.to_csv('../data/mergent_intellect_executives/AtoZ_executives_nlp.csv')
AtoZ_df

,ID,company_name,fullname,title,firstname,lastname,flag_executive,flag_owner,prob_nw_eth_wiki,prob_nw_eth_fl_reg_name,prob_nw_eth_fl_five_cat,prob_nw_eth_census_fn,prob_nw_eth_census_ln,prob_nw_eth_nc
0,2,Capital Jiu-Jitsu Team/Royce,Jeremiah Lafreniere,Owner,Jeremiah,Lafreniere,1,1,0.041037,0.392866,0.915747,0.613421,0.076179,0.229721
1,3,Integrated Clinical Concepts,Judith Wald,Manager,Judith,Wald,1,0,0.046589,0.039200,0.390792,0.145289,0.093963,0.616429
2,4,Magical Elves Inc,Ben Mack,Manager,Ben,Mack,1,0,0.048640,0.175978,0.679456,0.651104,0.584001,0.582545
3,5,Potomac River Running Store,Chris Jakubek,Manager,Chris,Jakubek,1,0,0.456253,0.117893,0.663747,0.301765,0.010478,0.633961
4,7,Atlantic Sweeping & Cleaning Inc,Bonita Tenicela,Controller,Bonita,Tenicela,1,0,0.214810,0.235418,0.712825,0.452071,0.963061,0.513625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29871,50399,Richard A Spagna,Richard Spagna,Manager,Richard,Spagna,1,0,NaN,NaN,NaN,NaN,NaN,NaN
29872,50400,Kathy Wolf MD PC,Kathy Wolf,Principal,Kathy,Wolf,1,0,NaN,NaN,NaN,NaN,NaN,NaN
29873,50401,Roger F Vorcheimer DDS,Roger Vorcheimer,Professional,Roger,Vorcheimer,1,0,NaN,NaN,NaN,NaN,NaN,NaN
29874,50402,Richard F Blackburn,Richard Blackburn,Manager,Richard,Blackburn,1,0,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
azdf =  pd.read_csv('../data/mergent_intellect_executives/AtoZ_executive_flagged.csv.xz')

azdf['prob_nw_eth_wiki'] = a['not_white']
azdf['prob_nw_eth_fl_reg_name'] = b['not_white']
azdf['prob_nw_eth_fl_five_cat'] = c['not_white']
azdf['prob_nw_eth_nc'] = e['not_white']

azdf.to_csv('../data/mergent_intellect_executives/AtoZ_executives_top4_nlp.csv')
azdf


,ID,company_name,fullname,title,firstname,lastname,flag_executive,flag_owner,prob_nw_eth_wiki,prob_nw_eth_fl_reg_name,prob_nw_eth_fl_five_cat,prob_nw_eth_nc
0,2,Capital Jiu-Jitsu Team/Royce,Jeremiah Lafreniere,Owner,Jeremiah,Lafreniere,1,1,0.041037,0.392866,0.915747,0.229721
1,3,Integrated Clinical Concepts,Judith Wald,Manager,Judith,Wald,1,0,0.046589,0.039200,0.390792,0.616429
2,4,Magical Elves Inc,Ben Mack,Manager,Ben,Mack,1,0,0.048640,0.175978,0.679456,0.582545
3,5,Potomac River Running Store,Chris Jakubek,Manager,Chris,Jakubek,1,0,0.456253,0.117893,0.663747,0.633961
4,7,Atlantic Sweeping & Cleaning Inc,Bonita Tenicela,Controller,Bonita,Tenicela,1,0,0.214810,0.235418,0.712825,0.513625
...,...,...,...,...,...,...,...,...,...,...,...,...
29871,50399,Richard A Spagna,Richard Spagna,Manager,Richard,Spagna,1,0,NaN,NaN,NaN,NaN
29872,50400,Kathy Wolf MD PC,Kathy Wolf,Principal,Kathy,Wolf,1,0,NaN,NaN,NaN,NaN
29873,50401,Roger F Vorcheimer DDS,Roger Vorcheimer,Professional,Roger,Vorcheimer,1,0,NaN,NaN,NaN,NaN
29874,50402,Richard F Blackburn,Richard Blackburn,Manager,Richard,Blackburn,1,0,NaN,NaN,NaN,NaN


### I didn't add the RaceBERT model yet to the dataframe only functions from ethnicolr

In [17]:
import pandas as pd
from racebert import RaceBERT
# Load the dataset

model = RaceBERT()
# Concatenate first and last names to form full names
namesdf['fullname'] = namesdf['firstname'] + ' ' + namesdf['lastname']
# Predict race for all names
predicted_race = model.predict_race(namesdf['fullname'])
# # Add the predicted race to the original DataFrame
#namesdf['predicted_race'] = predicted_race
# Create a new DataFrame to store the names and predicted races
result_df = pd.DataFrame({'fullname': namesdf['fullname'], 'predicted_race': predicted_race})
# # Split the ‘predicted_race’ column into ‘label’ and ‘score’ columns
result_df[['label', 'score']] = result_df['predicted_race'].apply(pd.Series)
# # Drop the ‘predicted_race’ column
result_df.drop('predicted_race', axis=1, inplace=True)

result_df
len(predicted_race)

/home/huz2ph/.conda/envs/myenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4552

In [18]:
not_wh = []

scorelist = list(result_df['score'])
labellist = list(result_df['label'])

for i in range(len(labellist)):
    l = labellist[i]
    p = scorelist[i]
    
    if 'nh_white' in l:
        not_wh.append(1 - p)
    else:
        not_wh.append(p)

print(len(not_wh))


4552


In [19]:
mergent_true_df['prob_nw_racebert_fl'] = not_wh
# len(not_wh)
empty = []
for t in list(mergent_true_df['firstname']):
    for u in list(results_df['fullname']):
        for g in not_wh:
                   if t in mergent_true_df[f]:
                      empty.append(g)
                   else:
                      empty.append(None)
                       
              
                
empty

#mergent_true_df.to_csv('../data/mergent_intellect_executives/mergent_intellect_executives_nlp.csv')

NameError: name 'mergent_true_df' is not defined